 ## HW4 - EuroSAT Land Use and Land Cover Classification using Deep Learning
 
In this homework your task is to implement deep learning models to solve a typical problem in satellite imaging  using a benchmark dataset. The homework was designed to make you work on increasingly more complex models. We hope that the homework will be very helpful to improve your skills and knowledge in deep learning!


### S1:

- Visit the EuroSAT data description page and download the data: https://github.com/phelber/eurosat

- Split the data into training (50%) and testing sets (50%), stratified on class labels (equal percentage of each class type in train and test sets).

- Convert each RGB image to grayscale and flatten the images into a data matrix (n x p: n = #samples, p = #pixels in each image)


In [70]:
import cv2
from PIL import Image
import PIL
import numpy as np
import pandas as pd
import os


import sys
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib
import matplotlib.pyplot as plt
from sklearn import cluster

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

from sklearn.utils import shuffle

from sklearn import preprocessing


from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten,Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
import numpy as np

In [71]:
DIRECTORY = r'EuroSAT\2750'

CATEGORIES = ['SeaLake', 'River','Residential','PermanentCrop',
              'Pasture','Industrial','Highway','HerbaceousVegetation',
             'Forest','AnnualCrop']
num_class=(len(CATEGORIES))
image=PIL.Image.open('EuroSAT/2750/Forest/Forest_1.jpg',mode='r', formats=None)
pix=image.load()
print(image.size)
w=image.size[0]
h=image.size[1]
n_spectrum = 2


(64, 64)


In [72]:
data_1= []
X_1=[]
Y_1=[]
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        new_arr = cv2.resize(arr, (64, 64))
        X_1.append(new_arr)
        Y_1.append(label)
        data_1.append([new_arr, label])

In [73]:
X_value_1=np.array(X_1)
X_value_1=X_value_1.reshape(X_value_1.shape[0],-1)
X_value_1=X_value_1.astype('float32')
X_value_1/=255
Y_value_1=np.array(Y_1)


In [74]:

x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(X_value_1, Y_value_1, stratify = Y_value_1, train_size = 0.5, random_state=42)
y_train_1 = to_categorical(y_train_1, num_class)
y_test_1 = to_categorical(y_test_1, num_class)

In [75]:
y_test_1

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [76]:

batch_size=(x_train_1.shape)
batch_size

(13500, 4096)

### S2:

- Implement a first deep learning model (M.1) using a fully connected network with a single fully connected layer (i.e: input layer + fully connected layer as the output layer). 

###  CNN:
1. reduce para: kernel sharing para, proximity in pixel, kernel is learned by the DNN

Q2.1: Calculate classification accuracy on the test data. 
0.17444443702697754

In [77]:
model_1 = Sequential()
model_1.add(Dense(num_class, activation='softmax', input_shape=(batch_size[1],)))
model_1.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 10)                40970     
                                                                 
Total params: 40,970
Trainable params: 40,970
Non-trainable params: 0
_________________________________________________________________


In [78]:
model_1.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [79]:
history = model_1.fit(x_train_1, y_train_1,
                    batch_size=64,
                      # unbiased sampling, but large the size the lower variance
                      # 32->64
                    epochs=50,
                    verbose=0,
                    validation_data=(x_test_1, y_test_1))

In [80]:
score = model_1.evaluate(x_test_1, y_test_1, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.196153163909912
Test accuracy: 0.263481467962265


### S3:

- Implement a second deep learning model (M.2) adding an additional fully connected hidden layer (with an arbitrary number of nodes) to the previous model. 


Q3.1: Calculate classification accuracy on the test data. 
0.29822221398353577 

In [81]:
model_2 = Sequential()
model_2.add(Dense(512, activation='relu', input_shape=(batch_size[1],)))
model_2.add(Dense(num_class, activation='softmax'))
model_2.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 512)               2097664   
                                                                 
 dense_27 (Dense)            (None, 10)                5130      
                                                                 
Total params: 2,102,794
Trainable params: 2,102,794
Non-trainable params: 0
_________________________________________________________________


In [82]:
model_2.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [64]:
history = model_2.fit(x_train_1, y_train_1,
                    batch_size=64,
                      # unbiased sampling, but large the size the lower variance
                      # 32->64
                    epochs=50,
                    verbose=0,
                    validation_data=(x_test_1, y_test_1))

In [65]:
score = model_2.evaluate(x_test_1, y_test_1, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.7196553945541382
Test accuracy: 0.3468148112297058


### S4:

- Implement a third deep learning model (M.3) adding two additional fully connected hidden layers (with arbitrary number of nodes) for a total of four, as well as drop-out layers to the previous model. 

#todo: how many layers in total

Q4.1: Calculate classification accuracy on the test data.

Q4.2: Compare against previous models. Which model was the "best"? Why?

In [66]:
# softmax: multi-class prob
# relu: positive value 
model_3 = Sequential()
model_3.add(Dense(512, activation='relu', input_shape=(batch_size[1],)))
model_3.add(Dense(216, activation='relu'))
model_3.add(Dense(64, activation='relu'))
model_3.add(Dropout(0.25))
model_3.add(Dense(num_class, activation='softmax'))
model_3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 512)               2097664   
                                                                 
 dense_22 (Dense)            (None, 216)               110808    
                                                                 
 dense_23 (Dense)            (None, 64)                13888     
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_24 (Dense)            (None, 10)                650       
                                                                 
Total params: 2,223,010
Trainable params: 2,223,010
Non-trainable params: 0
_________________________________________________________________


In [67]:
model_3.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [68]:
history = model_3.fit(x_train_1, y_train_1,
                    batch_size=64,

                    epochs=50,
                    verbose=0,
                    validation_data=(x_test_1, y_test_1))

In [69]:
score = model_3.evaluate(x_test_1, y_test_1, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.2947914600372314
Test accuracy: 0.1111111119389534


### S5:

- Take the original RGB images and do not vectorize them. Use these images as the data input for the following models (M.4 and M.5).
- Implement a fourth CNN model (M.4) that includes the following layers: Conv2D, MaxPooling2D, Dropout, Flatten, Dense. 

Q5.1: Calculate classification accuracy on the test data.

Q5.2: Compare against previous models. Which model was the "best"? Why?

In [83]:
data = []
X=[]
Y=[]
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_COLOR)
        new_arr = cv2.resize(arr, (64, 64))
        X.append(new_arr)
        Y.append(label)
        data.append([new_arr, label])

In [84]:
X=np.array(X)
X.shape

(27000, 64, 64, 3)

In [85]:

X_value=X.astype('float32')
X_value/=255
Y_value=np.array(Y)

x_train, x_test, y_train, y_test = train_test_split(X_value, Y_value, stratify = Y_value, train_size = 0.5, random_state=42)
y_train = to_categorical(y_train, num_class)
y_test = to_categorical(y_test, num_class)

input_shape=(64,64,3)

In [86]:
model_4 = Sequential()

model_4.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',input_shape=input_shape
                ))

model_4.add(MaxPooling2D(pool_size=(2, 2)))
model_4.add(Dropout(0.25))
model_4.add(Flatten())
model_4.add(Dense(num_class, activation='softmax'))
model_4.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 flatten_3 (Flatten)         (None, 30752)             0         
                                                                 
 dense_28 (Dense)            (None, 10)                307530    
                                                                 
Total params: 308,426
Trainable params: 308,426
Non-trainable params: 0
_______________________________________________

In [87]:
model_4.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [88]:
history = model_4.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model_4.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
211/211 [==============================] - 33s 149ms/step - loss: 1.6715 - accuracy: 0.3901 - val_loss: 1.3090 - val_accuracy: 0.4890
Epoch 2/10
211/211 [==============================] - 28s 134ms/step - loss: 1.2586 - accuracy: 0.5361 - val_loss: 1.1920 - val_accuracy: 0.5866
Epoch 3/10
211/211 [==============================] - 29s 138ms/step - loss: 1.1159 - accuracy: 0.6007 - val_loss: 1.1511 - val_accuracy: 0.5787
Epoch 4/10
211/211 [==============================] - 29s 139ms/step - loss: 1.0219 - accuracy: 0.6423 - val_loss: 0.9549 - val_accuracy: 0.6709
Epoch 5/10
211/211 [==============================] - 30s 141ms/step - loss: 0.9459 - accuracy: 0.6663 - val_loss: 0.9584 - val_accuracy: 0.6652
Epoch 6/10
211/211 [==============================] - 31s 145ms/step - loss: 0.8932 - accuracy: 0.6935 - val_loss: 0.9256 - val_accuracy: 0.6913
Epoch 7/10
211/211 [==============================] - 32s 153ms/step - loss: 0.8501 - accuracy: 0.7071 - val_loss: 0.9755 - val_ac

### S6:

- Using RGB images from S5, implement a fifth deep learning model (M.5) targeting accuracy that will outperform all previous models. You are free to use any tools and techniques, as well as pre-trained models for transfer learning. 

Q6.1: Describe the model you built, and why you chose it.

Q6.2: Calculate classification accuracy on the test data.

Q6.3: Compare against previous models. Which model was the "best"? Why?

Q6.4: What are the two classes with the highest labeling error? Explain using data and showing mis-classified examples.

In [89]:
model_5 = Sequential()
model_5.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(Dropout(0.25))
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(Flatten())
model_5.add(Dense(num_class, activation='softmax'))
model_5.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 12, 12, 64)      

In [90]:
model_5.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model_5.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model_5.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
211/211 [==============================] - 60s 279ms/step - loss: 1.6790 - accuracy: 0.3624 - val_loss: 1.2301 - val_accuracy: 0.5621
Epoch 2/10
211/211 [==============================] - 58s 274ms/step - loss: 1.2057 - accuracy: 0.5617 - val_loss: 1.0119 - val_accuracy: 0.6242
Epoch 3/10
211/211 [==============================] - 58s 276ms/step - loss: 1.0082 - accuracy: 0.6439 - val_loss: 1.5864 - val_accuracy: 0.4594
Epoch 4/10
211/211 [==============================] - 58s 276ms/step - loss: 0.8837 - accuracy: 0.6870 - val_loss: 0.9666 - val_accuracy: 0.6301
Epoch 5/10
211/211 [==============================] - 58s 276ms/step - loss: 0.7902 - accuracy: 0.7167 - val_loss: 0.6758 - val_accuracy: 0.7599
Epoch 6/10
211/211 [==============================] - 59s 278ms/step - loss: 0.7176 - accuracy: 0.7430 - val_loss: 0.8177 - val_accuracy: 0.6840
Epoch 7/10
211/211 [==============================] - 58s 276ms/step - loss: 0.6902 - accuracy: 0.7559 - val_loss: 0.6874 - val_ac

### S7:

- Apply your best model on multispectral images. You may use whichever image channels you wish, so long as you use more than just RGB (although you are not required to use any color channels).

Q7.1: Calculate classification accuracy on the test data.

Q7.2: Compare against results using RGB images.

In [91]:
DIRECTORY = r'EuroSATallBands\ds\images\remote_sensing\otherDatasets\sentinel_2\tif'

CATEGORIES = ['SeaLake', 'River','Residential','PermanentCrop',
              'Pasture','Industrial','Highway','HerbaceousVegetation',
             'Forest','AnnualCrop']
num_class=(len(CATEGORIES))


In [92]:
import tifffile

data_2= []
X_2=[]
Y_2=[]
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = tifffile.imread(img_path)
#         new_arr = cv2.resize(arr, (64, 64))
        X_2.append(arr)
        Y_2.append(label)
        data_2.append([arr, label])

In [93]:
X_2=np.array( X_2).astype('float32')
X_2.shape
X_value_2=X_2[:,:,:,:4]
X_value_2/=255
Y_value_2=np.array(Y_2)

x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(X_value_2, Y_value_2, stratify = Y_value, train_size = 0.5, random_state=42)
y_train_2 = to_categorical(y_train_2, num_class)
y_test_2 = to_categorical(y_test_2, num_class)

MemoryError: Unable to allocate 5.36 GiB for an array with shape (27000, 64, 64, 13) and data type float32

In [ ]:
X_value_2.shape

(27000, 64, 64, 4)

In [ ]:
model_6 = Sequential()
model_6.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 4)))
model_6.add(MaxPooling2D((2, 2)))
model_6.add(Dropout(0.25))
model_6.add(Conv2D(64, (3, 3), activation='relu'))
model_6.add(MaxPooling2D((2, 2)))
model_6.add(Conv2D(64, (3, 3), activation='relu'))
model_6.add(Flatten())
model_6.add(Dense(num_class, activation='softmax'))
model_6.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        1184      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 64)       

In [ ]:
y_test_2.shape

(13500, 10)

In [ ]:
model_6.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model_6.fit(x_train_2, y_train_2,
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test_2, y_test_2))

score = model_6.evaluate(x_test_2, y_test_2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
211/211 [==============================] - 59s 273ms/step - loss: 1.7025 - accuracy: 0.3830 - val_loss: 1.3741 - val_accuracy: 0.4957
Epoch 2/50
211/211 [==============================] - 59s 278ms/step - loss: 1.2455 - accuracy: 0.5614 - val_loss: 0.9978 - val_accuracy: 0.6413
Epoch 3/50
211/211 [==============================] - 67s 316ms/step - loss: 1.0517 - accuracy: 0.6334 - val_loss: 1.5797 - val_accuracy: 0.4689
Epoch 4/50
211/211 [==============================] - 57s 267ms/step - loss: 0.9282 - accuracy: 0.6730 - val_loss: 0.8766 - val_accuracy: 0.7003
Epoch 5/50
211/211 [==============================] - 58s 273ms/step - loss: 0.8429 - accuracy: 0.7022 - val_loss: 0.9003 - val_accuracy: 0.6786
Epoch 6/50
211/211 [==============================] - 62s 294ms/step - loss: 0.7632 - accuracy: 0.7341 - val_loss: 0.8292 - val_accuracy: 0.6991
Epoch 7/50
211/211 [==============================] - 60s 284ms/step - loss: 0.7218 - accuracy: 0.7556 - val_loss: 0.7354 - val_ac